# 数字识别-颜色轨迹识别


* 操作说明：
    选择你想识别的HSV颜色阈值，然后用这个颜色的物体在dachbot摄像头前面画出数字，
    
    当dachbot识别到颜色后就会保存你画出的轨迹，然后既可以对你画出的数字进行识别了

![title](other_data/01.jpg)

## 1.导入所需模块

In [ ]:
from nxbot import Robot,event,bgr8_to_jpeg
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F
import time
import cv2
import numpy as np
from PIL import Image
from collections import deque
from IPython.display import display
import ipywidgets
import traitlets
import ipywidgets.widgets as widgets
import net

## 2.实例化模型

In [ ]:
net = net.Net()

## 3.加载模型

In [ ]:
'''
加载模型
'''
# 选择默认的模型或者自己训练的模型
model_path = r'../../../models/local/personal_net/digit_classification.pth'
# model_path = 'studens_models/MNIST_student.pth'
model = torch.load(model_path)
if torch.cuda.is_available():
    model = model.cuda()

## 4.数据预处理

In [ ]:
def preprogress(img):
    # 将图像resize为28*28
    newImage = cv2.resize(img, (28, 28))
    # cv2格式转为np格式
    newImage = np.array(newImage)
    newImage = newImage.reshape(28, 28, 1)
    # 将图像数据转为tensor，并且进行标准化。
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
    newImage = transform(newImage)
    # 将数组增加一维，并转换为模型接收的数据格式
    newImage = torch.unsqueeze(newImage, 0).cuda().float()
    return newImage

## 5.检测模型是否能正常使用

我们通过numpy创建与我们将要预测的图片格式一致的形状为（28，28，1）的数组，这里我们创建的全为1的数组将这个数组经过预处理再将数据放入模型中，如果能运行通过说明模型可以正常使用了。

In [ ]:
try:
    img_data = np.ones([28,28,1],np.float32)
    model(preprogress(img_data)).detach().half().cpu().numpy().flatten()
except:
    print('请检查模型是否正确')

## 6.定义预测函数

In [ ]:

'''
通过模型预测结果
'''
def predict(alphabet):
    newImage = preprogress(alphabet)
    with torch.no_grad():
        # 将数据放入模型
        out = model(newImage)
        # 找出预测概率最大的结果
        prob, numbers = torch.max(out, 1)
        if prob > 0.85:
            number = numbers.item()
    return number

## 7.设置HSV颜色阈值

* 找到指定HSV颜色值，并记录该颜色运动轨迹，最后将轨迹灰度图输入模型进行预测。

HSV颜色分量范围
一般对颜色空间的图像进行有效处理都是在HSV空间进行的，然后对于基本色中对应的HSV分量需要给定一个严格的范围，下面是通过实验计算的模糊范围。

H:  0— 180

S:  0— 255

V:  0— 255

此处把部分红色归为紫色范围：

![title](other_data/04.png) 

![title](other_data/03.png)  ![title](other_data/02.png)

In [ ]:
# 红色HSV颜色体系取值范围
Lower_red = np.array([157, 43, 43])
Upper_red = np.array([172, 255, 255])

## 8.创建显示窗口

In [ ]:
image_widget = widgets.Image(format='jpeg', width=300, height=300)
blackboard_widget = widgets.Image(format='jpeg', width=300, height=300)

## 9.定义预测函数

In [ ]:
global number, points, kernel, blackboard
number = ''
points = deque(maxlen=512)
kernel = np.ones((3, 3), np.uint8)
blackboard = np.zeros((480, 640, 3), dtype=np.uint8)


def draw(img):
    
    global number, points, kernel, blackboard
    # 在图像中找出红色
    # 图像镜像翻转
    img = cv2.flip(img, 1)
    # RGB转HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # 红色HSV阈值
    redMask = cv2.inRange(hsv, Lower_red, Upper_red)
    # 图像腐蚀
    redMask = cv2.erode(redMask, kernel, iterations=2)
    # 图像膨胀
    redMask = cv2.dilate(redMask, kernel, iterations=2)
    redMask=cv2.GaussianBlur(redMask,(3,3),0)
    # cnts是列个list，list中每个元素都是图像中的一个轮廓，用numpy中的ndarray表示
    cnts = cv2.findContours(redMask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[0]
    center = None
#     if cnts!= None:
        # 如果找到红色就在周围画圆，用队列记录所有红点圆圈力矩。
    if len(cnts) > 0:
        cnt = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(cnt)
        cv2.circle(img, (int(x), int(y)), int(radius), (0, 255, 255), 2)

        # *计算轮廓力矩（质心）
        M = cv2.moments(cnt)
        center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
        points.appendleft(center)

    # 如果没有找到红点，将记录的力矩点添加到空矩阵中然后通过预处理再进行识别。
    elif len(cnts) == 0:
        if len(points) != 0:
            # 转为灰度图
            blackboard_gray = cv2.cvtColor(blackboard, cv2.COLOR_BGR2GRAY)
            # 中值滤波
            blur1 = cv2.medianBlur(blackboard_gray, 15)
            # 高斯滤波
            blur1 = cv2.GaussianBlur(blur1, (5, 5), 0)
            # 设置轮廓阈值
            thresh1 = cv2.threshold(blur1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
            # cnts是列个list，list中每个元素都是图像中的一个轮廓，用numpy中的ndarray表示
            cnts = cv2.findContours(thresh1.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[0]
            # 如果轮廓数量大于等于1。
            if len(cnts) >= 1:
                # 找出轮廓中占图像区域最大的一个轮廓
                cnt = max(cnts, key=cv2.contourArea)
                # 如果超过1000
                if cv2.contourArea(cnt) > 1000:
                    # 用一个最小的矩形，把找到的形状包起来，返回矩形左上角坐标与宽高
                    x, y, w, h = cv2.boundingRect(cnt)
                    # 将这个矩形区域转为灰度图
                    alphabet = blackboard_gray[y:y + h, x:x + w]
                    # 开始预测
                    number = predict(alphabet)
            # 初始化points， blackboard，清除画板。
            points = deque(maxlen=512)
            blackboard = np.zeros((480, 640, 3), dtype=np.uint8)

    # 画出找到的连续红点力矩。
    for i in range(1, len(points)):
        if points[i - 1] is None or points[i] is None:
            continue
        # 在图像中显示
        cv2.line(img, points[i - 1], points[i], (255, 0, 0), 12)
        cv2.line(blackboard, points[i - 1], points[i], (255, 255, 255), 12)
        
    
        
    cv2.putText(img, "number is: " + str(number), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
    image_widget.value = bgr8_to_jpeg(img)
    blackboard_widget.value = bgr8_to_jpeg(blackboard)

## 10.使用线程说出预测结果

In [ ]:
import threading
threading_stop=False
def interaction():
    while threading_stop==False:
        global number
        if number!='':
            rbt.speech.play_text('预测结果是数字{}'.format(number), True)
            number=''
        time.sleep(1)

## 11.开始预测

In [ ]:
def on_new_image(evt):
    # 获取图像数据
    img = evt.dict['data']
    draw(img)

rbt = Robot()
rbt.connect()    
rbt.camera.start()
rbt.base.set_ptz(20)
rbt.event_manager.add_event_listener(event.EventTypes.NEW_CAMERA_IMAGE,on_new_image)
rbt.speech.start()

process1 = threading.Thread(target=interaction,)
process1.start()

display(ipywidgets.HBox([image_widget, blackboard_widget]))

# 12.断开机器人连接

In [ ]:
# threading_stop=True
# rbt.disconnect()